In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

In [4]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 500
USE_CUDA = torch.cuda.is_available()
#DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)


다음 기기로 학습합니다: cuda


In [2]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [3]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Generator Code
#nc - number of color channels in the input images. For color images this is 3
#nz - length of latent vector (i.e. size of generator input)
#ngf - relates to the depth of feature maps carried through the generator (Size of feature maps in generator)
#ndf - sets the depth of feature maps propagated through the discriminator (Size of feature maps in discriminator)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

nc=1 #channel이랄 것이 딱히 없음


class DCGANGenerator(nn.Module):
    def __init__(self):
        super(DCGANGenerator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, (2,4), (1,0), bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 16
            
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False, dilation =(1,2)),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 35
            nn.ConvTranspose2d( ngf , 1, 4, 2, (2,0), bias=False, dilation=(1,2)),
            nn.Tanh()
            # state size. (ngf) x 32 x 75
            
        )
    
    
    def forward(self, input):
        return self.main(input)

In [88]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            #300*75
            nn.Conv2d(1, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf, ndf * 2, (8,4), 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 2, ndf * 4, (8,4), 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 4, ndf * 8, (8,4), (4,2), 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(ndf * 8, 1,(4, 4), (5,1), 0, bias=False),
            
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [115]:
# Root directory for dataset
dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Generator Code
#nc - number of color channels in the input images. For color images this is 3
#nz - length of latent vector (i.e. size of generator input)
#ngf - relates to the depth of feature maps carried through the generator (Size of feature maps in generator)
#ndf - sets the depth of feature maps propagated through the discriminator (Size of feature maps in discriminator)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

nc=1 #channel이랄 것이 딱히 없음


class DCGANGenerator(nn.Module):
    def __init__(self):
        super(DCGANGenerator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, (2,4), (1,0), bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 16
            
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, (2,1), bias=False, dilation =(1,2)),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 35
            nn.ConvTranspose2d( ngf , 1, 4, 2, (1,0), bias=False, dilation=(1,2)),
            nn.Tanh()
            # state size. (ngf) x 32 x 75
            
        )
    
    
    def forward(self, input):
        return self.main(input)

In [116]:
model=DCGANGenerator()
disc=Discriminator()
#real_cpu = data[0].to(device)
#b_size = real_cpu.size(0)""
b_size=3  #batch size
noise=torch.randn(1, 100, 35, 1)
noise.size()
out=model(noise)
out.shape

torch.Size([1, 1, 300, 75])

In [90]:
input_data=torch.rand(1,1,300,75)
out=disc(input_data)

In [91]:
out.shape

torch.Size([1, 1, 1, 1])

In [92]:
print(out)

tensor([[[[0.4531]]]], grad_fn=<SigmoidBackward>)


In [118]:
for i in range(0, 25):
    if i %3==0:
        print(i)
        model.load_state_dict(torch.load('./snapshot/cnngan300/1106_cnn_'+str(i)+'.pt'))
        output=model(torch.tensor(noise))
        np.save('./snapshot/cnngan300/1106_cnn_'+str(i)+'.npy', output.detach().numpy())

0
3
6
9


<ipython-input-118-86d3e16f2c24>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output=model(torch.tensor(noise))


12
15
18
21
24


torch.Size([1, 100, 1, 1])

input tensor를 forward()함수에 넣을 때 여러 개의 input이 batch로 들어갈 수 있다. 

<ipython-input-48-33249b6f5399>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output=model(torch.tensor(noise))


In [49]:
output.size()

torch.Size([1, 1, 30, 75])

In [50]:
output

tensor([[[[ 0.1498,  0.0000, -0.2855,  ..., -0.0297,  0.0000,  0.0964],
          [ 0.2644,  0.0000, -0.2206,  ..., -0.1783,  0.0000,  0.0104],
          [ 0.1498,  0.0000, -0.2263,  ..., -0.1562,  0.0000,  0.0964],
          ...,
          [ 0.2644,  0.0000, -0.1704,  ..., -0.2621,  0.0000,  0.0104],
          [ 0.1498,  0.0000, -0.1481,  ..., -0.3720,  0.0000,  0.0964],
          [ 0.2644,  0.0000, -0.0091,  ..., -0.4380,  0.0000,  0.0104]]]],
       grad_fn=<TanhBackward>)

In [13]:
arr.shape

NameError: name 'arr' is not defined